In [30]:
import pandas as pd
from streaming_optimizer import optimize_streaming_packages
from datetime import datetime

In [31]:
game_raw = pd.read_csv('data/bc_game.csv') # some games and when they are played
streaming_packages_raw = pd.read_csv('data/bc_streaming_package.csv') # streaming packages with price
streaming_offers_raw = pd.read_csv('data/bc_streaming_offer.csv')  # which service offers which game, live? on-demand-highlights?
streaming_packages_raw.head(3)

,id,name,monthly_price_cents,monthly_price_yearly_subscription_in_cents
0,2,MagentaTV - MegaSport,NaN,6000
1,3,ZDF - Free-TV,0.0,0
2,4,ARD - Free-TV,0.0,0


In [4]:
from streaming_optimizer import optimize_streaming_packages
from datetime import datetime, timedelta

# Example Input Data
packages = ["P1", "P2"]
games = ["G1", "G2", "G3", "G4", "G5", 'G6', 'G7']
game_dates = {
    "G1": datetime(2023, 1, 15),
    "G2": datetime(2023, 2, 20),
    "G3": datetime(2023, 3, 25),
    "G4": datetime(2023, 4, 10),
    "G5": datetime(2023, 4, 20),
    "G6": datetime(2024, 1, 10),
    "G7": datetime(2024, 3, 15),
}
C_month = {'P1': 20}
C_year = {'P1': 180}
P_g = {'G1': ['P1'], 'G2': ['P1'], 'G3': ['P1', 'P2'], 'G4': ['P1', 'P2'], 'G5': ['P1', 'P2'],
    'G6': ['P1'], 'G7': ['P1', 'P2']}
optimize_streaming_packages(packages, games, game_dates, C_month, C_year, P_g)



Status: Optimal
Total Cost: 100.0
Active Monthly Subscriptions:
  Package: P1, Start Date: 2023-01-15
  Package: P1, Start Date: 2023-02-20
  Package: P1, Start Date: 2023-03-25
  Package: P1, Start Date: 2024-01-10
  Package: P1, Start Date: 2024-03-15
Active Yearly Subscriptions:
